In [1]:
import tensorflow as tf
import numpy as np
from  keras.utils import np_utils
from matplotlib import pyplot as plt
import pandas as pd
import os 

In [2]:
print(tf.__version__)

2.3.0


In [3]:
def generate_train_header(x, y, file='tain_data.h'):
    '''
    this method generate the
    :param x:  input x data size
    :param y:  input label (one hot label)
    :return:
    '''
    # quantize input x
    min_value = np.min(x)
    max_value = np.max(x)

    int_bits = int(np.ceil(np.log2(max(abs(min_value), abs(max_value)))))
    dec_bits = 7 - int_bits
    x = np.round(x*2**dec_bits).astype(np.int8)
    
#     print(x[0][0])
    data = x.astype(dtype="byte")
    label = y
    node = 0
    with open(file, 'w') as f:
        num_of_image = x.shape[0]
        
        print("Number of training images:",num_of_image)
        for i in range(num_of_image):
            f.write('#define NODE_%d_TRAIN_IMG_%d {'%(node,i) )
            (data[i]).flatten().tofile(f, sep=", ") # convert 0~1 to 0~127
            f.write('} \n')
            f.write('#define NODE_%d_TRAIN_IMG%d_LABEL'% (node, i))
            f.write(' %d \n \n' % label[i])
        f.write('#define NODE_%d_TOTAL_TRAIN_IMAGES %d \n \n'%(node,num_of_image))

        f.write('static q7_t NODE_%d_TRAIN_IMAGES[%d][%d] = {' % (node,num_of_image, data[0].flatten().shape[0]))
        f.write('NODE_%d_TRAIN_IMG_0'%(node))
        for i in range(num_of_image -1):
            f.write(',NODE_%d_TRAIN_IMG_%d'%(node, i+1))
        f.write('};\n\n')

        f.write('static q7_t NODE_%d_TRAIN_LABELS[%d] = {' % (node,num_of_image))
        f.write('NODE_%d_TRAIN_IMG0_LABEL'%(node))
        for i in range(num_of_image -1):
            f.write(',NODE_%d_TRAIN_IMG%d_LABEL'%(node, i+1))
        f.write('};\n\n')

#     return 000


In [4]:
def generate_test_bin(x, y, file='test_data.h'):
    '''
    method to generate the test data as a header file
    :param x:  input x data size
    :param y:  input label 
    :write to file 
    :return: nothing
    '''
    # quantize input x
    min_value = np.min(x)
    max_value = np.max(x)

    int_bits = int(np.ceil(np.log2(max(abs(min_value), abs(max_value)))))
    dec_bits = 7 - int_bits
    x = np.round(x*2**dec_bits).astype(np.int8)
    
    data = x
    label = y

    with open(file, 'w') as f:
        num_of_image = x.shape[0]
        print("Number of test images:",num_of_image)
        for i in range(num_of_image):
            f.write('#define TEST_IMG%d {'% (i))
            data[i].flatten().tofile(f, sep=", ")
            f.write('} \n')
            f.write('#define TEST_IMG%d_LABEL'% (i))
            f.write(' %d \n \n' % label[i])
        f.write('#define TOTAL_TEST_IMAGES %d \n \n'%(num_of_image))

        f.write('static q7_t TEST_IMAGES[%d][%d] = {' % (num_of_image, data[0].flatten().shape[0]))
        f.write('TEST_IMG0')
        for i in range(num_of_image -1):
            f.write(',TEST_IMG%d'%(i+1))
        f.write('};\n\n')

        f.write('static q7_t TEST_LABELS[%d] = {' % (num_of_image))
        f.write('TEST_IMG0_LABEL')
        for i in range(num_of_image -1):
            f.write(',TEST_IMG%d_LABEL'%(i+1))
        f.write('};\n\n')


## Cifar images

In [ ]:


cifar10 = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()


mean = np.mean(x_train)
stddev = np.std(x_train)

x_train = (x_train - mean ) / stddev
x_test =  (x_test-mean) /stddev

# shuffle test/train data
permutation = np.random.permutation(x_test.shape[0])
x_test = x_test[permutation, :]
y_test = y_test[permutation]
permutation = np.random.permutation(x_train.shape[0])
x_train = x_train[permutation, :]
y_train = y_train[permutation]


for i in range (20):
    


    
    labelindices = np.random.choice(10, 3, replace=False)
    print(labelindices)

    # generate_train_header(x_train[:1500], y_train[:1500], file='cifar_train_data.h')
    # generate_test_bin(x_test[:1200], y_test[:1200], file='cifar_test_data.h')


    subset_x_train = x_train[np.isin(y_train, labelindices).flatten()]
    subset_y_train = y_train[np.isin(y_train, labelindices).flatten()]
    # subset_y_train, _ = pd.factorize(np.concatenate( subset_y_train, axis=0 ))  

    subset_y_train = np.unique(subset_y_train, return_inverse=True)[1] 


    subset_x_test = x_test[np.isin(y_test, labelindices).flatten()]
    subset_y_test = y_test[np.isin(y_test, labelindices).flatten()] 


    subset_y_test = np.unique(subset_y_test, return_inverse=True)[1] 
    # generate_test_bin(subset_x_test, subset_y_test, file='cifar_test_cmsis.h')


    # subset_y_test, _ = pd.factorize(np.concatenate( subset_y_test, axis=0 ))  


    # subset_x_train = x_train[np.isin(y_train, [7,8,9]).flatten()] 
    # subset_y_train = y_train[np.isin(y_train, [7,8,9]).flatten()]- 7

    # subset_x_test = x_test[np.isin(y_test, [7,8,9]).flatten()]
    # subset_y_test = y_test[np.isin(y_test, [7,8,9]).flatten()]- 7

    indices = np.random.choice(subset_x_train.shape[0], 600, replace=False)

    #print(subset_x_train.shape[0])

    subset_x_train = subset_x_train[indices]
    subset_y_train= subset_y_train[indices]
    

    generate_train_header(subset_x_train, subset_y_train, file='./src/cifar_train_data.h')
    generate_test_bin(subset_x_test, subset_y_test, file='./src/cifar_test_data.h')


    # generate_train_header(x_train ,y_train, file='cifar_train_data.h')
    # generate_test_bin(x_test, y_test, file='cifar_test_data.h')


    # generate_train_header(subset_x_train, subset_y_train, file='cifar_train_data.h')
    # subset_x_train.shape[0]
    # federated_train_images(0, subset_x_train*127, subset_y_train, subset_x_train.shape[0], file='CIFAR3-TrainSet.h')

    # subset_x_test.shape[0]

    # imagesTestcfile(subset_x_test*127, subset_y_test,subset_x_test.shape[0] , file='CIFAR3-TestSet.h')


    print ("done")
    !make 
    !./MiniLearn

[9 7 2]
Number of training images: 600
Number of test images: 3000
done
mkdir -p obj
gcc -Iinclude/ -Isrc/  -I../CMSIS/CMSIS/DSP/Include/ -I../CMSIS/CMSIS/Core/Include -I../CMSIS/CMSIS/NN/Include  -Wall -Wextra -Wno-unknown-pragmas -Wshift-overflow -ggdb3 -fPIC -g -O2 -g -c ./src/main.c -o obj/main.o
./src/main.c:483:14: warning: unused variable 'blockSize' [-Wunused-variable]
    uint32_t blockSize = sizeof(y_out);
             ^
1 warning generated.
gcc -Iinclude/ -Isrc/  -I../CMSIS/CMSIS/DSP/Include/ -I../CMSIS/CMSIS/Core/Include -I../CMSIS/CMSIS/NN/Include  -Wall -Wextra -Wno-unknown-pragmas -Wshift-overflow -ggdb3 -fPIC -g -O2 -g -c ./src/image.c -o obj/image.o
In file included from ./src/image.c:47:
src/stb_image.h:6361:84: warning: unused parameter 'req_comp'
      [-Wunused-parameter]
  ...*s, stbi__gif *g, int *comp, int req_comp, stbi_uc *two_back)
                                      ^
src/stb_image.h:6585:107: warning: unused parameter 'ri' [-Wunused-parameter]
  ...*s, in

ar rcs libMiniLearn.a obj/main.o obj/image.o obj/args.o obj/matrix.o obj/list.o obj/data.o obj/classifier.o obj/net.o obj/maxpool_layer.o obj/convolutional_layer.o obj/connected_layer.o obj/activations.o obj/arm_fully_connected_q7_opt.o obj/arm_convolve_HWC_q7_basic.o obj/arm_convolve_HWC_q7_fast.o obj/arm_relu_q7.o obj/arm_pool_q7_HWC.o obj/arm_convolve_HWC_q7_RGB.o obj/arm_softmax_q7.o obj/batchnorm_layer.o obj/activation_layer.o
/Library/Developer/CommandLineTools/usr/bin/ranlib: file: libMiniLearn.a(activations.o) has no symbols
gcc -Iinclude/ -Isrc/  -I../CMSIS/CMSIS/DSP/Include/ -I../CMSIS/CMSIS/Core/Include -I../CMSIS/CMSIS/NN/Include  -Wall -Wextra -Wno-unknown-pragmas -Wshift-overflow -ggdb3 -fPIC -g -O2 -g obj/main.o obj/image.o obj/args.o obj/matrix.o obj/list.o obj/data.o obj/classifier.o obj/net.o obj/maxpool_layer.o obj/convolutional_layer.o obj/connected_layer.o obj/activations.o obj/arm_fully_connected_q7_opt.o obj/arm_convolve_HWC_q7_basic.o obj/arm_convolve_HWC_q7_fas

gcc -Iinclude/ -Isrc/  -I../CMSIS/CMSIS/DSP/Include/ -I../CMSIS/CMSIS/Core/Include -I../CMSIS/CMSIS/NN/Include  -Wall -Wextra -Wno-unknown-pragmas -Wshift-overflow -ggdb3 -fPIC -g -O2 -g -c ./src/connected_layer.c -o obj/connected_layer.o
gcc -Iinclude/ -Isrc/  -I../CMSIS/CMSIS/DSP/Include/ -I../CMSIS/CMSIS/Core/Include -I../CMSIS/CMSIS/NN/Include  -Wall -Wextra -Wno-unknown-pragmas -Wshift-overflow -ggdb3 -fPIC -g -O2 -g -c ./src/activations.c -o obj/activations.o
gcc -Iinclude/ -Isrc/  -I../CMSIS/CMSIS/DSP/Include/ -I../CMSIS/CMSIS/Core/Include -I../CMSIS/CMSIS/NN/Include  -Wall -Wextra -Wno-unknown-pragmas -Wshift-overflow -ggdb3 -fPIC -g -O2 -g -c ../CMSIS/CMSIS/NN/Source/FullyConnectedFunctions/arm_fully_connected_q7_opt.c -o obj/arm_fully_connected_q7_opt.o
gcc -Iinclude/ -Isrc/  -I../CMSIS/CMSIS/DSP/Include/ -I../CMSIS/CMSIS/Core/Include -I../CMSIS/CMSIS/NN/Include  -Wall -Wextra -Wno-unknown-pragmas -Wshift-overflow -ggdb3 -fPIC -g -O2 -g -c ../CMSIS/CMSIS/NN/Source/Convolution

12 :: train acc = 0.910000 & test acc = 0.843000
13 :: train acc = 0.913333 & test acc = 0.842333
14 :: train acc = 0.913333 & test acc = 0.842000
15 :: train acc = 0.915000 & test acc = 0.842000
16 :: train acc = 0.915000 & test acc = 0.841333
17 :: train acc = 0.918333 & test acc = 0.840667
18 :: train acc = 0.920000 & test acc = 0.840667
19 :: train acc = 0.920000 & test acc = 0.841000
20 :: train acc = 0.921667 & test acc = 0.841000
21 :: train acc = 0.921667 & test acc = 0.840333
22 :: train acc = 0.921667 & test acc = 0.840667
23 :: train acc = 0.925000 & test acc = 0.841333
24 :: train acc = 0.930000 & test acc = 0.841000
[1 4 3]
Number of training images: 600
Number of test images: 3000
done
gcc -Iinclude/ -Isrc/  -I../CMSIS/CMSIS/DSP/Include/ -I../CMSIS/CMSIS/Core/Include -I../CMSIS/CMSIS/NN/Include  -Wall -Wextra -Wno-unknown-pragmas -Wshift-overflow -ggdb3 -fPIC -g -O2 -g -c ./src/main.c -o obj/main.o
./src/main.c:483:14: warning: unused variable 'blockSize' [-Wunused-variab

ar rcs libMiniLearn.a obj/main.o obj/image.o obj/args.o obj/matrix.o obj/list.o obj/data.o obj/classifier.o obj/net.o obj/maxpool_layer.o obj/convolutional_layer.o obj/connected_layer.o obj/activations.o obj/arm_fully_connected_q7_opt.o obj/arm_convolve_HWC_q7_basic.o obj/arm_convolve_HWC_q7_fast.o obj/arm_relu_q7.o obj/arm_pool_q7_HWC.o obj/arm_convolve_HWC_q7_RGB.o obj/arm_softmax_q7.o obj/batchnorm_layer.o obj/activation_layer.o
/Library/Developer/CommandLineTools/usr/bin/ranlib: file: libMiniLearn.a(activations.o) has no symbols
gcc -Iinclude/ -Isrc/  -I../CMSIS/CMSIS/DSP/Include/ -I../CMSIS/CMSIS/Core/Include -I../CMSIS/CMSIS/NN/Include  -Wall -Wextra -Wno-unknown-pragmas -Wshift-overflow -ggdb3 -fPIC -g -O2 -g obj/main.o obj/image.o obj/args.o obj/matrix.o obj/list.o obj/data.o obj/classifier.o obj/net.o obj/maxpool_layer.o obj/convolutional_layer.o obj/connected_layer.o obj/activations.o obj/arm_fully_connected_q7_opt.o obj/arm_convolve_HWC_q7_basic.o obj/arm_convolve_HWC_q7_fas

...

12 :: train acc = 0.800000 & test acc = 0.691333
13 :: train acc = 0.803333 & test acc = 0.693333
14 :: train acc = 0.808333 & test acc = 0.692333
15 :: train acc = 0.813333 & test acc = 0.693667
16 :: train acc = 0.813333 & test acc = 0.695000
17 :: train acc = 0.820000 & test acc = 0.695000
18 :: train acc = 0.820000 & test acc = 0.694000
19 :: train acc = 0.823333 & test acc = 0.695000
20 :: train acc = 0.826667 & test acc = 0.694667
21 :: train acc = 0.826667 & test acc = 0.694333
22 :: train acc = 0.828333 & test acc = 0.693333
23 :: train acc = 0.828333 & test acc = 0.694667
24 :: train acc = 0.828333 & test acc = 0.694667
[7 8 4]
Number of training images: 600
Number of test images: 3000
done
gcc -Iinclude/ -Isrc/  -I../CMSIS/CMSIS/DSP/Include/ -I../CMSIS/CMSIS/Core/Include -I../CMSIS/CMSIS/NN/Include  -Wall -Wextra -Wno-unknown-pragmas -Wshift-overflow -ggdb3 -fPIC -g -O2 -g -c ./src/main.c -o obj/main.o
./src/main.c:483:14: warning: unused variable 'blockSize' [-Wunused-variab

4 warnings generated.
gcc -I../CMSIS/CMSIS/DSP/Include/ -I../CMSIS/CMSIS/Core/Include -I../CMSIS/CMSIS/NN/Include  -Wall -Wextra -Wno-unknown-pragmas -Wshift-overflow -ggdb3 -fPIC -g -O2 -g -shared obj/main.o obj/image.o obj/args.o obj/matrix.o obj/list.o obj/data.o obj/classifier.o obj/net.o obj/maxpool_layer.o obj/convolutional_layer.o obj/connected_layer.o obj/activations.o obj/arm_fully_connected_q7_opt.o obj/arm_convolve_HWC_q7_basic.o obj/arm_convolve_HWC_q7_fast.o obj/arm_relu_q7.o obj/arm_pool_q7_HWC.o obj/arm_convolve_HWC_q7_RGB.o obj/arm_softmax_q7.o obj/batchnorm_layer.o obj/activation_layer.o -o libMiniLearn.so -lm -pthread 
ar rcs libMiniLearn.a obj/main.o obj/image.o obj/args.o obj/matrix.o obj/list.o obj/data.o obj/classifier.o obj/net.o obj/maxpool_layer.o obj/convolutional_layer.o obj/connected_layer.o obj/activations.o obj/arm_fully_connected_q7_opt.o obj/arm_convolve_HWC_q7_basic.o obj/arm_convolve_HWC_q7_fast.o obj/arm_relu_q7.o obj/arm_pool_q7_HWC.o obj/arm_convolv

gcc -Iinclude/ -Isrc/  -I../CMSIS/CMSIS/DSP/Include/ -I../CMSIS/CMSIS/Core/Include -I../CMSIS/CMSIS/NN/Include  -Wall -Wextra -Wno-unknown-pragmas -Wshift-overflow -ggdb3 -fPIC -g -O2 -g -c ./src/maxpool_layer.c -o obj/maxpool_layer.o
gcc -Iinclude/ -Isrc/  -I../CMSIS/CMSIS/DSP/Include/ -I../CMSIS/CMSIS/Core/Include -I../CMSIS/CMSIS/NN/Include  -Wall -Wextra -Wno-unknown-pragmas -Wshift-overflow -ggdb3 -fPIC -g -O2 -g -c ./src/convolutional_layer.c -o obj/convolutional_layer.o
gcc -Iinclude/ -Isrc/  -I../CMSIS/CMSIS/DSP/Include/ -I../CMSIS/CMSIS/Core/Include -I../CMSIS/CMSIS/NN/Include  -Wall -Wextra -Wno-unknown-pragmas -Wshift-overflow -ggdb3 -fPIC -g -O2 -g -c ./src/connected_layer.c -o obj/connected_layer.o
gcc -Iinclude/ -Isrc/  -I../CMSIS/CMSIS/DSP/Include/ -I../CMSIS/CMSIS/Core/Include -I../CMSIS/CMSIS/NN/Include  -Wall -Wextra -Wno-unknown-pragmas -Wshift-overflow -ggdb3 -fPIC -g -O2 -g -c ./src/activations.c -o obj/activations.o
gcc -Iinclude/ -Isrc/  -I../CMSIS/CMSIS/DSP/Incl

2 :: train acc = 0.753333 & test acc = 0.707333
3 :: train acc = 0.785000 & test acc = 0.725000
4 :: train acc = 0.805000 & test acc = 0.734333
5 :: train acc = 0.825000 & test acc = 0.738667
6 :: train acc = 0.833333 & test acc = 0.743333
7 :: train acc = 0.843333 & test acc = 0.745000
8 :: train acc = 0.853333 & test acc = 0.749667
9 :: train acc = 0.858333 & test acc = 0.750667
10 :: train acc = 0.866667 & test acc = 0.755333
11 :: train acc = 0.871667 & test acc = 0.758667
12 :: train acc = 0.878333 & test acc = 0.760667
13 :: train acc = 0.876667 & test acc = 0.764000
14 :: train acc = 0.881667 & test acc = 0.768000
15 :: train acc = 0.885000 & test acc = 0.770000
16 :: train acc = 0.886667 & test acc = 0.770333
17 :: train acc = 0.886667 & test acc = 0.770333
18 :: train acc = 0.890000 & test acc = 0.769333
19 :: train acc = 0.895000 & test acc = 0.770333
20 :: train acc = 0.896667 & test acc = 0.769667
21 :: train acc = 0.903333 & test acc = 0.770000
22 :: train acc = 0.908333 &

./src/activation_layer.c:98:45: warning: unused parameter 'rate'
      [-Wunused-parameter]
void update_activation_layer(layer l, float rate, float momentum, float decay){}
                                            ^
./src/activation_layer.c:98:57: warning: unused parameter 'momentum'
      [-Wunused-parameter]
void update_activation_layer(layer l, float rate, float momentum, float decay){}
                                                        ^
./src/activation_layer.c:98:73: warning: unused parameter 'decay'
      [-Wunused-parameter]
void update_activation_layer(layer l, float rate, float momentum, float decay){}
                                                                        ^
4 warnings generated.
gcc -I../CMSIS/CMSIS/DSP/Include/ -I../CMSIS/CMSIS/Core/Include -I../CMSIS/CMSIS/NN/Include  -Wall -Wextra -Wno-unknown-pragmas -Wshift-overflow -ggdb3 -fPIC -g -O2 -g -shared obj/main.o obj/image.o obj/args.o obj/matrix.o obj/list.o obj/data.o obj/classifier.o obj/net.o obj/

In [ ]:
print ("done")


In [ ]:
for i in range (21):
    label3random = np.random.choice(10, 3, replace=False)
    label3random
    subset_x_test = x_test[np.isin(y_test, label3random).flatten()]
    subset_y_test = y_test[np.isin(y_test, label3random).flatten()] 
    # generate_test_bin(subset_x_test, subset_y_test, file='cifar_test_cmsis.h')


    # subset_y_test, _ = pd.factorize(np.concatenate( subset_y_test, axis=0 ))  


    # subset_y_test = np.unique(subset_y_test, return_inverse=True)[1] 



    generate_test_bin(subset_x_test, subset_y_test, file='./src/cifar_test_data.h')
    print("done")
    !make 
    !./MiniLearn

In [ ]:
# subset_y_test = y_test[np.isin(y_test, labelindices).flatten()] 
# values = np.array([8,2,1,2,8])
# a = np.concatenate( subset_y_test, axis=0 )
# codes, uniques = pd.factorize(a)  # default: na_sentinel=-1

In [ ]:
subset_y_test

In [ ]:
print(subset_x_train.shape)
print(subset_y_test.shape)
print (np.unique(subset_y_test))

In [ ]:
# imgplot = plt.imshow(subset_x_train[21])
# print(subset_y_train[21])